In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_circuit_layout, circuit_drawer

In [2]:
# Create a bloated circuit with 4 qubits
qc = QuantumCircuit(4)

# Redundant gates and inefficient layout
qc.h(0)
qc.h(0)  # Cancels out with previous H
qc.cx(0, 1)
qc.cx(1, 0)  # Reversible CX pair
qc.swap(0, 2)
qc.swap(2, 0)  # Double swap = identity
qc.rx(0.5, 3)
qc.rx(-0.5, 3)  # Cancels out
qc.rz(0.25, 1)
qc.rz(0.25, 1)  # Can be merged
qc.cx(2, 3)
qc.cx(3, 2)  # Reversible again
qc.measure_all()

# Transpile with aggressive optimization
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend, optimization_level=3)

print("Original Inefficient Circuit:")
print(qc.draw())
print("Transpiled Optimized Circuit:")
print(transpiled_qc.draw())

# Compare metrics
print("\n📊 Improvement Summary:")
print(f"Original Depth: {qc.depth()} → Transpiled Depth: {transpiled_qc.depth()}")
print(f"Original Gate Count: {qc.size()} → Transpiled Gate Count: {transpiled_qc.size()}")
print("\nGate Type Breakdown:")
print("Original:", qc.count_ops())
print("Transpiled:", transpiled_qc.count_ops())

Original Inefficient Circuit:
           ┌───┐      ┌───┐         ┌───┐                                    ░ »
   q_0: ───┤ H ├──────┤ H ├──────■──┤ X ├─X──────────────X───────────────────░─»
           └───┘      └───┘    ┌─┴─┐└─┬─┘ │ ┌──────────┐ │ ┌──────────┐      ░ »
   q_1: ───────────────────────┤ X ├──■───┼─┤ Rz(0.25) ├─┼─┤ Rz(0.25) ├──────░─»
                               └───┘      │ └──────────┘ │ └──────────┘┌───┐ ░ »
   q_2: ──────────────────────────────────X──────────────X──────■──────┤ X ├─░─»
        ┌─────────┐┌──────────┐                               ┌─┴─┐    └─┬─┘ ░ »
   q_3: ┤ Rx(0.5) ├┤ Rx(-0.5) ├───────────────────────────────┤ X ├──────■───░─»
        └─────────┘└──────────┘                               └───┘          ░ »
meas: 4/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌─┐         
«   q_0: ┤M├─────────
«        └╥┘┌─┐      
«   q_1: ─╫─┤

In [3]:
# unitary synthesis

from qiskit.transpiler.passes import UnitarySynthesis
from qiskit.transpiler import PassManager
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.unitary([[0, 1], [1, 0]], [0])  # Example unitary

pm = PassManager([UnitarySynthesis(basis_gates=['u3', 'cx'])])
optimized_qc = pm.run(qc)

print(qc.draw())
print(optimized_qc.draw())

     ┌─────────┐
q_0: ┤ Unitary ├
     └─────────┘
q_1: ───────────
                
global phase: π
     ┌────────────┐
q_0: ┤ U3(π,-π,0) ├
     └────────────┘
q_1: ──────────────
                   


In [4]:
import trace
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2

# Create a bloated circuit with 4 qubits
qc = QuantumCircuit(4)

# Redundant gates and inefficient layout
qc.h(0)
qc.h(0)  # Cancels out with previous H
qc.cx(0, 1)
qc.cx(1, 0)  # Reversible CX pair
qc.swap(0, 2)
qc.swap(2, 0)  # Double swap = identity
qc.rx(0.5, 3)
qc.rx(-0.5, 3)  # Cancels out
qc.rz(0.25, 1)
qc.rz(0.25, 1)  # Can be merged
qc.cx(2, 3)
qc.cx(3, 2)  # Reversible again
qc.measure_all()

# Transpile with aggressive optimization
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend, optimization_level=3)

tracer = trace.Trace(
    trace=True,
    count=False,
    ignoredirs=['/usr', '/lib', '/site-packages'],  # Ignore system libraries
)

# tracer.runfunc(transpile, qc, backend=FakeJakarta(), optimization_level=3)

with open("transpile_trace_output.txt", "w") as f:
    with redirect_stdout(f):
        tracer.runfunc(transpile, qc, backend=FakeJakartaV2(), optimization_level=3)

NameError: name 'redirect_stdout' is not defined

In [6]:
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes import Unroll3qOrMore
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_circuit_layout
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel

# 1. Create a quantum circuit with gates outside the target basis
qc = QuantumCircuit(2)
qc.h(0)
qc.swap(0, 1)
qc.ry(1.23, 0)
qc.measure_all()

print("Original Circuit:")
print(qc)

# 2. Define the target basis gates
target_basis = ['cx', 'u'] # 'u' is a single-qubit unitary gate

# 3. Create a PassManager and add BasisTranslator
# For multi-qubit gates like swap, you may need other passes to break them down first
# Unroll3qOrMore helps with multi-qubit gates that have a basis translation.
pm = PassManager([Unroll3qOrMore(), BasisTranslator(sel,target_basis=target_basis)])

# 4. Run the pass manager on the circuit
translated_circuit = pm.run(qc)

# 5. Print the translated circuit
print("\nTranslated Circuit:")
print(translated_circuit)

# You can also use Qiskit's built-in transpilation for a more complete workflow
# Note: For versions of Qiskit newer than 1.0, BasisTranslator is used by default
# inside the transpiler, so you may not need to call it directly for simple cases.
# from qiskit import transpile
#
# # This will use the built-in BasisTranslator as part of the pipeline
# built_in_transpiled_circuit = transpile(qc, basis_gates=['u', 'cx'])
# print("\nBuilt-in transpiled circuit:")
# print(built_in_transpiled_circuit)


Original Circuit:
        ┌───┐   ┌──────────┐ ░ ┌─┐   
   q_0: ┤ H ├─X─┤ Ry(1.23) ├─░─┤M├───
        └───┘ │ └──────────┘ ░ └╥┘┌─┐
   q_1: ──────X──────────────░──╫─┤M├
                             ░  ║ └╥┘
meas: 2/════════════════════════╩══╩═
                                0  1 

Translated Circuit:
        ┌────────────┐     ┌───┐     ┌─────────────┐ ░ ┌─┐   
   q_0: ┤ U(π/2,0,π) ├──■──┤ X ├──■──┤ U(1.23,0,0) ├─░─┤M├───
        └────────────┘┌─┴─┐└─┬─┘┌─┴─┐└─────────────┘ ░ └╥┘┌─┐
   q_1: ──────────────┤ X ├──■──┤ X ├────────────────░──╫─┤M├
                      └───┘     └───┘                ░  ║ └╥┘
meas: 2/════════════════════════════════════════════════╩══╩═
                                                        0  1 
